In [1]:
import ee
import numpy as np
from osgeo import gdal
from osgeo import osr
import time
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import pandas as pd
import seaborn as sns
from scipy import stats
import folium
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg')
from matplotlib import rcParams
import matplotlib.cm as cm # matplotlib's color map library
import warnings

warnings.filterwarnings("ignore")

# init the ee object
ee.Initialize()

C:\Users\owner\anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  from ipykernel import kernelapp as app


In [2]:
# https://www.techforwildlife.com/blog/2019/1/22/analysing-drone-and-satellite-imagery-using-vegetation-indices
def p2r(point, area_size = 500):
    dist_lat = abs(area_size / 110.540)/1000
    dist_lon = abs(area_size / (111.320 * np.cos(point[0])))/1000
    area = {"type": "FeatureCollection", "features": [{"type": "Feature", "properties": {},"geometry": {"type": "Polygon", "coordinates": [[[point[0] - dist_lon/2, point[1] - dist_lat/2], [point[0] + dist_lon/2, point[1] - dist_lat/2], [point[0] + dist_lon/2, point[1] + dist_lat/2], [point[0] - dist_lon/2, point[1] + dist_lat/2], [point[0] - dist_lon/2, point[1] - dist_lat/2]]]}}]}
    coords_area = area['features'][0]['geometry']['coordinates']
    aoi_area = ee.Geometry.Polygon(coords_area)
    return area, aoi_area

def terrain(df):
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    ax.plot_trisurf(df['Y'], df['X'], df['Z'], cmap=plt.cm.viridis, linewidth=0.2)
    surf=ax.plot_trisurf(df['Y'], df['X'], df['Z'], cmap=plt.cm.viridis, linewidth=0.2)
    fig.colorbar(surf, shrink=0.5, aspect=5)
    ax.view_init(azim = 45, elev = 45)

def terrain_3d(a, e):
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    ax.plot_trisurf(df['Y'], df['X'], df['Z'], cmap=plt.cm.viridis, linewidth=0.2)
    surf=ax.plot_trisurf(df['Y'], df['X'], df['Z'], cmap=plt.cm.viridis, linewidth=0.2)
    fig.colorbar(surf, shrink=0.5, aspect=5)
    ax.view_init(azim = a, elev = e)
    
def contour_plt(df, no_con):
    Z = df.pivot_table(index='X', columns='Y', values='Z').T.values

    X_unique = np.sort(df.X.unique())
    Y_unique = np.sort(df.Y.unique())
    X, Y = np.meshgrid(X_unique, Y_unique)

    # Initialize plot objects
    rcParams['figure.figsize'] = 5, 5 # sets plot size
    fig = plt.figure()
    ax = fig.add_subplot(111)

    # Define levels in z-axis where we want lines to appear
    levels = np.linspace(np.min(df['Z']), np.max(df['Z']), no_con)

    # Generate a color mapping of the levels we've specified
    cpf = ax.contourf(X, Y, Z, len(levels), cmap = cm.Reds)

    # Set all level lines to black
    line_colors = ['black' for l in cpf.levels]

    # Make plot and customize axes
    cp = ax.contour(X, Y, Z, levels=levels, colors=line_colors)
    ax.clabel(cp, fontsize=10, colors=line_colors)
    plt.xticks()
    plt.yticks()
    ax.set_xlabel('X-axis')
    _ = ax.set_ylabel('Y-axis')
    plt.grid()

In [3]:
areaJson, area = p2r([23.0043, 40.5241], area_size = 500)
area = ee.Geometry.Rectangle([40.5241, 23.0043, 40.5241 + 0.01, 23.0043 + 0.01])

In [4]:
def elevations(area):
    img1 = ee.Image('USGS/SRTMGL1_003') \
  .select(['elevation'])
    new=img1.toArray().toArray(1)

    band_arrs = img1.sampleRectangle(region=area)
    band_arr_b2 = band_arrs.get('elevation')
    np_arr_b2 = np.array(band_arr_b2.getInfo())

    latlon = ee.Image.pixelLonLat().addBands(img1)
    latlon = latlon.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=area,
      scale=100);
    lats = np.array((ee.Array(latlon.get("latitude")).getInfo()))
    lons = np.array((ee.Array(latlon.get("longitude")).getInfo()))
    return np_arr_b2, lats, lons

np_arr_b2, lats, lons = elevations(area)
# np_arr_b2 = np.where(np_arr_b2 > 968, np_arr_b2, np.nan)
df = pd.DataFrame(np_arr_b2).unstack().reset_index()
df.columns = ["X","Y","Z"]
 
df['X'] = pd.Categorical(df['X'])
df['X'] = df['X'].cat.codes